# Contract Review Workflow

*   Updated by: Wilfredo Aaron Sosa Ramos
*   Credits to: https://github.com/run-llama/llamacloud-demo/blob/main/examples/document_workflows/contract_review/contract_review.ipynb

![](contract_review.png)

This tutorial shows you how to create an agentic workflow that can review a contract for compliance with certain regulations. We will parse the contract into a set of key clauses, match it with relevant clauses from a guideline repository (here, we specifically do GDPR), and then produce a compliance summary.

In [1]:
!pip install -q llama-index llama-index-indices-managed-llama-cloud llama-cloud llama-parse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.8 MB/s eta 0:00:00


In [2]:
!pip install -q llama-index-utils-workflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.8 MB/s eta 0:00:00


In [3]:
import nest_asyncio

nest_asyncio.apply()

## Setup

We setup an index for guidelines. In this case it's just the GDPR document.

We also setup our parser.

In [4]:
!mkdir -p data
!wget "https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:32016R0679" -O data/gdpr.pdf

--2024-12-25 17:40:16--  https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:32016R0679
Resolving eur-lex.europa.eu (eur-lex.europa.eu)... 52.85.132.127, 52.85.132.11, 52.85.132.12, ...
Connecting to eur-lex.europa.eu (eur-lex.europa.eu)|52.85.132.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘data/gdpr.pdf’

data/gdpr.pdf           [ <=>                ] 959.27K  --.-KB/s    in 0.01s   

2024-12-25 17:40:16 (69.7 MB/s) - ‘data/gdpr.pdf’ saved [982296]



### Setup Index
Here we use LlamaCloud: https://cloud.llamaindex.ai/. If you don't have access yet, you're always welcome to use our open-source VectorStoreIndex.

In [5]:
!pip install --upgrade llama-index-core

In [7]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

Option 2 - LlamaCloud:

In [ ]:
# option 1
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex

index = LlamaCloudIndex(
  name="gdpr",
  project_name="llamacloud_demo",
  organization_id="cdcb3478-1348-492e-8aa0-25f47d1a3902",
  # api_key="llx-..."
)

retriever = index.as_retriever(similarity_top_k=2)

Option 2 - Open Source Index:

In [8]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
# Load documents and build index
documents = SimpleDirectoryReader(
    "/content/data"
).load_data()
index = VectorStoreIndex.from_documents(documents)

In [9]:
retriever = index.as_retriever(similarity_top_k=3)

In [10]:
!pwd

/content


### Setup Parser

Here we use LlamaParse to parse the vendor agremeent.

In [12]:
llama_parse_api_key = userdata.get('LLAMA_CLOUD_API_KEY')

In [13]:
from llama_parse import LlamaParse

# use our multimodal models for extractions
parser = LlamaParse(result_type="markdown", api_key=llama_parse_api_key)

### Define Contract Output Schema

We want to extract relevant clauses from the agreement in order to match it against relevant clauses in the GDPR. This schema defines a way to structuring the set of extracted clauses.

In [14]:
from typing import List, Optional
from pydantic import BaseModel, Field

class ContractClause(BaseModel):
    clause_text: str = Field(..., description="The exact text of the clause.")
    mentions_data_processing: bool = Field(False, description="True if the clause involves personal data collection or usage.")
    mentions_data_transfer: bool = Field(False, description="True if the clause involves transferring personal data, especially to third parties or across borders.")
    requires_consent: bool = Field(False, description="True if the clause explicitly states that user consent is needed for data activities.")
    specifies_purpose: bool = Field(False, description="True if the clause specifies a clear purpose for data handling or transfer.")
    mentions_safeguards: bool = Field(False, description="True if the clause mentions security measures or other safeguards for data.")

class ContractExtraction(BaseModel):
    vendor_name: Optional[str] = Field(None, description="The vendor's name if identifiable.")
    effective_date: Optional[str] = Field(None, description="The effective date of the agreement, if available.")
    governing_law: Optional[str] = Field(None, description="The governing law of the contract, if stated.")
    clauses: List[ContractClause] = Field(..., description="List of extracted clauses and their relevant indicators.")

### Define Compliance Check Schema

Define a schema that matches clauses with relevant guidelines in GDPR.

In [15]:
from typing import Optional
from pydantic import BaseModel, Field

class GuidelineMatch(BaseModel):
    guideline_text: str = Field(..., description="The single most relevant guideline excerpt related to this clause.")
    similarity_score: float = Field(..., description="Similarity score indicating how closely the guideline matches the clause, e.g., between 0 and 1.")
    relevance_explanation: Optional[str] = Field(None, description="Brief explanation of why this guideline is relevant.")

class ClauseComplianceCheck(BaseModel):
    clause_text: str = Field(..., description="The exact text of the clause from the contract.")
    matched_guideline: Optional[GuidelineMatch] = Field(None, description="The most relevant guideline extracted via vector retrieval.")
    compliant: bool = Field(..., description="Indicates whether the clause is considered compliant with the referenced guideline.")
    notes: Optional[str] = Field(None, description="Additional commentary or recommendations.")

### Define Final Output Schema

This is the schema for the final compliance report. It contains the vendor name, if it's overall compliant, and also the summary notes.

It will be inferred from the individual checks for every clause.

In [16]:
from typing import Optional, List
from pydantic import BaseModel, Field

class ComplianceReport(BaseModel):
    vendor_name: Optional[str] = Field(None, description="The vendor's name if identified from the contract.")
    overall_compliant: bool = Field(..., description="Indicates if the contract is considered overall compliant.")
    summary_notes: Optional[str] = Field(None, description="General summary or recommendations for achieving full compliance.")

## Setup Contract Review Workflow

Let's define the following contract review workflow:
1. Extract out structured data from the vendor agreement.
2. For each clause, do retrieval against GDPR to see if it's compliant with guidelines.
3. Generate a final summary.

In [17]:
from llama_index.core.workflow import (
    Context,
)

print(dir(Context))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_deserialize_globals', '_deserialize_queue', '_serialize_globals', '_serialize_queue', 'collect_events', 'data', 'from_dict', 'get', 'get_result', 'lock', 'mark_in_progress', 'remove_from_in_progress', 'send_event', 'session', 'set', 'streaming_queue', 'to_dict', 'write_event_to_stream']


In [18]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Context,
    Workflow,
    step,
)
from llama_index.core.llms import LLM
from typing import Optional
from pydantic import BaseModel
from llama_index.core import SimpleDirectoryReader
from llama_index.core.schema import Document
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.prompts import ChatPromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.retrievers import BaseRetriever
from pathlib import Path
import logging
import json
import os

_logger = logging.getLogger(__name__)
_logger.setLevel(logging.INFO)


CONTRACT_EXTRACT_PROMPT = """\
You are given contract data below. \
Please extract out relevant information from the contract into the defined schema - the schema is defined as a function call.\

{contract_data}
"""

CONTRACT_MATCH_PROMPT = """\
Given the following contract clause and the corresponding relevant guideline text, evaluate the compliance \
and provide a JSON object that matches the ClauseComplianceCheck schema.

**Contract Clause:**
{clause_text}

**Matched Guideline Text(s):**
{guideline_text}
"""


COMPLIANCE_REPORT_SYSTEM_PROMPT = """\
You are a compliance reporting assistant. Your task is to generate a final compliance report \
based on the results of clause compliance checks against \
a given set of guidelines.

Analyze the provided compliance results and produce a structured report according to the specified schema.
Ensure that if there are no noncompliant clauses, the report clearly indicates full compliance.
"""

COMPLIANCE_REPORT_USER_PROMPT = """\
A set of clauses within a contract were checked against GDPR compliance guidelines for the following vendor: {vendor_name}.
The set of noncompliant clauses are given below.

Each section includes:
- **Clause:** The exact text of the contract clause.
- **Guideline:** The relevant GDPR guideline text.
- **Compliance Status:** Should be `False` for noncompliant clauses.
- **Notes:** Additional information or explanations.

{compliance_results}

Based on the above compliance results, generate a final compliance report following the `ComplianceReport` schema below.
If there are no noncompliant clauses, the report should indicate that the contract is fully compliant.
"""


class ContractExtractionEvent(Event):
    contract_extraction: ContractExtraction


class MatchGuidelineEvent(Event):
    clause: ContractClause


class MatchGuidelineResultEvent(Event):
    result: ClauseComplianceCheck


class GenerateReportEvent(Event):
    match_results: List[ClauseComplianceCheck]


class LogEvent(Event):
    msg: str
    delta: bool = False


class ContractReviewWorkflow(Workflow):
    """Contract review workflow."""

    def __init__(
        self,
        parser: LlamaParse,
        guideline_retriever: BaseRetriever,
        llm: LLM | None = None,
        similarity_top_k: int = 20,
        output_dir: str = "data_out",
        **kwargs,
    ) -> None:
        """Init params."""
        super().__init__(**kwargs)

        self.parser = parser
        self.guideline_retriever = guideline_retriever

        self.llm = llm or OpenAI(model="gpt-4o-mini")
        self.similarity_top_k = similarity_top_k

        # if not exists, create
        out_path = Path(output_dir) / "workflow_output"
        if not out_path.exists():
            out_path.mkdir(parents=True, exist_ok=True)
            os.chmod(str(out_path), 0o0777)
        self.output_dir = out_path

    @step
    async def parse_contract(
        self, ctx: Context, ev: StartEvent
    ) -> ContractExtractionEvent:
        # load output template file
        contract_extraction_path = Path(
            f"{self.output_dir}/contract_extraction.json"
        )
        if contract_extraction_path.exists():
            if self._verbose:
                ctx.write_event_to_stream(LogEvent(msg=">> Loading contract from cache"))
            contract_extraction_dict = json.load(open(str(contract_extraction_path), "r"))
            contract_extraction = ContractExtraction.model_validate(contract_extraction_dict)
        else:
            if self._verbose:
                ctx.write_event_to_stream(LogEvent(msg=">> Reading contract"))

            # no need to parse contract, it's already in markdown
            # you can use LlamaParse to parse more complex PDFs + other docs

            docs = SimpleDirectoryReader(input_files=[ev.contract_path]).load_data()

            # extract from contract
            prompt = ChatPromptTemplate.from_messages([
                ("user", CONTRACT_EXTRACT_PROMPT)
            ])
            contract_extraction = await llm.astructured_predict(
                ContractExtraction,
                prompt,
                contract_data="\n".join([d.get_content(metadata_mode="all") for d in docs])
            )
            if not isinstance(contract_extraction, ContractExtraction):
                raise ValueError(f"Invalid extraction from contract: {contract_extraction}")
            # save output template to file
            with open(contract_extraction_path, "w") as fp:
                fp.write(contract_extraction.model_dump_json())
        if self._verbose:
            ctx.write_event_to_stream(LogEvent(msg=f">> Contract data: {contract_extraction.dict()}"))

        return ContractExtractionEvent(contract_extraction=contract_extraction)

    @step
    async def dispatch_guideline_match(
        self, ctx: Context, ev: ContractExtractionEvent
    ) -> MatchGuidelineEvent:
        """For each clause in the contract, find relevant guidelines.

        Use a map-reduce pattern.

        """
        await ctx.set("num_clauses", len(ev.contract_extraction.clauses))
        await ctx.set("vendor_name", ev.contract_extraction.vendor_name)

        for clause in ev.contract_extraction.clauses:
            ctx.send_event(MatchGuidelineEvent(clause=clause, vendor_name=ev.contract_extraction.vendor_name))

    @step
    async def handle_guideline_match(
        self, ctx: Context, ev: MatchGuidelineEvent
    ) -> MatchGuidelineResultEvent:
        """Handle matching clause against guideline."""

        # retrieve matching guideline
        query = f"""\
Please find the relevant guideline from {ev.vendor_name} that aligns with the following contract clause:

{ev.clause.clause_text}
"""
        guideline_docs = self.guideline_retriever.retrieve(query)
        guideline_text="\n\n".join([g.get_content() for g in guideline_docs])
        if self._verbose:
            ctx.write_event_to_stream(
                LogEvent(msg=f">> Found guidelines: {guideline_text[:200]}...")
            )

        # extract from contract
        prompt = ChatPromptTemplate.from_messages([
            ("user", CONTRACT_MATCH_PROMPT)
        ])
        compliance_output = await llm.astructured_predict(
            ClauseComplianceCheck,
            prompt,
            clause_text=ev.clause.model_dump_json(),
            guideline_text=guideline_text

        )

        if not isinstance(compliance_output, ClauseComplianceCheck):
            raise ValueError(f"Invalid compliance check: {compliance_output}")

        return MatchGuidelineResultEvent(result=compliance_output)

    @step
    async def gather_guideline_match(
        self, ctx: Context, ev: MatchGuidelineResultEvent
    ) -> GenerateReportEvent:
        """Handle matching clause against guideline."""
        num_clauses = await ctx.get("num_clauses")
        events = ctx.collect_events(ev, [MatchGuidelineResultEvent] * num_clauses)
        if events is None:
            return

        match_results = [e.result for e in events]
        # save match results
        match_results_path = Path(
            f"{self.output_dir}/match_results.jsonl"
        )
        with open(match_results_path, "w") as fp:
            for mr in match_results:
                fp.write(mr.model_dump_json() + "\n")


        return GenerateReportEvent(match_results=[e.result for e in events])

    @step
    async def generate_output(
        self, ctx: Context, ev: GenerateReportEvent
    ) -> StopEvent:
        if self._verbose:
            ctx.write_event_to_stream(LogEvent(msg=">> Generating Compliance Report"))

        # if all clauses are compliant, return a compliant result
        non_compliant_results = [r for r in ev.match_results if not r.compliant]

        # generate compliance results string
        result_tmpl = """
1. **Clause**: {clause}
2. **Guideline:** {guideline}
3. **Compliance Status:** {compliance_status}
4. **Notes:** {notes}
"""
        non_compliant_strings = []
        for nr in non_compliant_results:
            non_compliant_strings.append(
                result_tmpl.format(
                    clause=nr.clause_text,
                    guideline=nr.matched_guideline.guideline_text,
                    compliance_status=nr.compliant,
                    notes=nr.notes
                )
            )
        non_compliant_str = "\n\n".join(non_compliant_strings)

        prompt = ChatPromptTemplate.from_messages([
            ("system", COMPLIANCE_REPORT_SYSTEM_PROMPT),
            ("user", COMPLIANCE_REPORT_USER_PROMPT)
        ])
        compliance_report = await llm.astructured_predict(
            ComplianceReport,
            prompt,
            compliance_results=non_compliant_str,
            vendor_name=await ctx.get("vendor_name")
        )

        return StopEvent(result={"report": compliance_report, "non_compliant_results": non_compliant_results})

In [19]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o")
workflow = ContractReviewWorkflow(
    parser=parser,
    guideline_retriever=retriever,
    llm=llm,
    verbose=True,
    timeout=None,  # don't worry about timeout to make sure it completes
)

#### Visualize the workflow

In [20]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(ContractReviewWorkflow, filename="contract_workflow.html")

<class 'NoneType'>
<class '__main__.MatchGuidelineEvent'>
<class '__main__.GenerateReportEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class '__main__.MatchGuidelineResultEvent'>
<class '__main__.ContractExtractionEvent'>
contract_workflow.html


## Run the Workflow

Let's run the full workflow and generate the output!

In [22]:
from IPython.display import clear_output

handler = workflow.run(contract_path="data/vendor_agreement.md")
async for event in handler.stream_events():
    if isinstance(event, LogEvent):
        if event.delta:
            print(event.msg, end="")
        else:
            print(event.msg)

response_dict = await handler
print(str(response_dict["report"]))

Running step parse_contract
>> Reading contract


<ipython-input-18-4f5a784640a7>:158: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  ctx.write_event_to_stream(LogEvent(msg=f">> Contract data: {contract_extraction.dict()}"))


Step parse_contract produced event ContractExtractionEvent
>> Contract data: {'vendor_name': 'ACME Office Supply, Inc.', 'effective_date': 'January 1, 2024', 'governing_law': 'Ireland', 'clauses': [{'clause_text': 'Vendor shall process Personal Data only: - To fulfill orders and manage deliveries - To provide customer support services - To maintain business records - To comply with legal obligations', 'mentions_data_processing': True, 'mentions_data_transfer': False, 'requires_consent': False, 'specifies_purpose': True, 'mentions_safeguards': False}, {'clause_text': 'Vendor shall: - Respond to data subject requests within 30 days - Provide data in a structured, commonly used format - Implement measures to facilitate data portability - Assist with data subject rights requests at no additional cost', 'mentions_data_processing': True, 'mentions_data_transfer': False, 'requires_consent': False, 'specifies_purpose': False, 'mentions_safeguards': False}, {'clause_text': '- Vendor maintains p

In [23]:
print(str(response_dict["report"]))

vendor_name='ACME Office Supply, Inc.' overall_compliant=False summary_notes='The contract contains noncompliant clauses regarding subprocessors and data transfer mechanisms. It lacks provisions for imposing data protection obligations on subprocessors and does not address liability issues. Additionally, it does not mention additional safeguards or compliance with standard contractual clauses for data transfers, nor does it address notification or consent for data transfers to new locations. Recommendations include revising these clauses to ensure compliance with GDPR guidelines.'


In [24]:
response_dict["non_compliant_results"]

[ClauseComplianceCheck(clause_text='- Vendor may engage subprocessors without prior Client approval - Subprocessors may be located in any jurisdiction globally - Notice of new subprocessors provided within 30 days of engagement - Client has no right to object to new subprocessors', matched_guideline=GuidelineMatch(guideline_text="Where a processor engages another processor for carrying out specific processing activities on behalf of the controller, the same data protection obligations as set out in the contract or other legal act between the controller and the processor as referred to in paragraph 3 shall be imposed on that other processor by way of a contract or other legal act under Union or Member State law, in particular providing sufficient guarantees to implement appropriate technical and organisational measures in such a manner that the processing will meet the requirements of this Regulation. Where that other processor fails to fulfil its data protection obligations, the initia

In [25]:
response_dict

{'report': ComplianceReport(vendor_name='ACME Office Supply, Inc.', overall_compliant=False, summary_notes='The contract contains noncompliant clauses regarding subprocessors and data transfer mechanisms. It lacks provisions for imposing data protection obligations on subprocessors and does not address liability issues. Additionally, it does not mention additional safeguards or compliance with standard contractual clauses for data transfers, nor does it address notification or consent for data transfers to new locations. Recommendations include revising these clauses to ensure compliance with GDPR guidelines.'),
 'non_compliant_results': [ClauseComplianceCheck(clause_text='- Vendor may engage subprocessors without prior Client approval - Subprocessors may be located in any jurisdiction globally - Notice of new subprocessors provided within 30 days of engagement - Client has no right to object to new subprocessors', matched_guideline=GuidelineMatch(guideline_text="Where a processor enga